# Gretel Bartender
Happy hour rolling around and you're tired of the same old stuff? Give
Gretel Bartender a try! Gretel Bartender uses Gretel GPT, a GPT-3 based model
to generate exciting new cocktails! While we can't guarantee they'll all be
good, they will all be created through the power of AI and Synthetic Data.

You can run the following notebook below to generate your own Synthetic Data
cocktail recipes. You can sign up for a free Gretel account today to try
this out at https://console.gretel.cloud 



## Generating Synthetic Text

This notebook will walk you through generating synthetic natural language text, similar to the data that you provide to it. This is accomplished by fine-tuning a large scale language model that has been pre-trained on billions of documents and is therefore capable of introducing realistic new variations into the data.
 
To run this notebook, you will need an [API key](https://docs.gretel.ai/environment-setup) from the [Gretel Console](https://console.gretel.ai).

** **Limitations and Biases** **
Large-scale language models such as Gretel GPT may produce untrue and/or offensive content without warning. We recommend having a human curate or filter the outputs before releasing them, both to censor undesirable content and to improve the quality of the results. For more information and examples please see [OpenAI](https://huggingface.co/gpt2#limitations-and-bias) and [EleutherAI](https://huggingface.co/EleutherAI/gpt-neo-125M#limitations-and-biases)'s docs for more details.

## Configure the project
* Install dependencies
* Import libraries
* Log into Gretel and set up a project

In [ ]:
%%capture
!pip install -U gretel-client

In [ ]:
import json

import pandas as pd
from gretel_client import configure_session
from gretel_client.helpers import poll
from gretel_client.projects import create_or_get_unique_project, get_project

pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Log into Gretel

configure_session(api_key="prompt", cache="yes", endpoint="https://api.gretel.cloud", validate=True, clear=True) #clear=True

project = create_or_get_unique_project(name="synthetic-text")
project

Gretel Api Key··········
Caching Gretel config to disk.
Using endpoint https://api.gretel.cloud
Logged in as mason@gretel.ai ✅


## Create the model configuration

In this notebook we will use GPT-Neo, a transformer model designed using EleutherAI's replication of OpenAI's GPT-3 Architecture. This model has been pre-trained on the Pile, a large-scale dataset using 300 billion tokens over 572,300 steps. In this introductory example, we will fine-tune GPT-Neo to generate synthetic (and hopefully entertaining) cocktail recipes by fine-tuning across a dataset of well known cocktail recipes. 

In [ ]:
config = {
  "schema_version": 1,
  "models": [
    {
      "gpt_x": {
        "data_source": "__",
        "pretrained_model": "EleutherAI/gpt-neo-125M",
        "batch_size": 4,
        "epochs": 3,
        "weight_decay": 0.1,
        "warmup_steps": 100,
        "lr_scheduler": "cosine",
        "learning_rate": 0.0005
      }
    }
  ]
}

print(json.dumps(config, indent=2))

{
  "schema_version": 1,
  "models": [
    {
      "gpt_x": {
        "data_source": "__",
        "pretrained_model": "EleutherAI/gpt-neo-125M",
        "batch_size": 4,
        "epochs": 3,
        "weight_decay": 0.1,
        "warmup_steps": 100,
        "lr_scheduler": "cosine",
        "learning_rate": 0.0005
      }
    }
  ]
}


## Load and preview the text dataset
Specify a data source to train the model on. This can be a local file, web location, or HDFS file. Currently, the text dataset must be saved in single-column CSV format.


In [ ]:
dataset_path = 'https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/drink-recipes.csv'
df = pd.concat([pd.read_csv(dataset_path)] * 1)

df.to_csv('training_data.csv', index=False)
df

,recipe
0,"This is called the Brainteaser. 30 ml white of Sambuca, 30 ml of Erin Cream, 5 ml of Advocaat. layered erin first, then sambuca and then avocart(should sit in middle of other two. To drink: use a straw to suck up avocart then shot the rest and then suck fumes up through straw."
1,"This is called the Porto flip. 3 parts of Brandy, 9 parts of Port, 2 parts of Egg Yolk. Shake ingredients together in a mixer with ice. Strain into glass, garnish and serve."
2,"This is called the Planter's Punch. 1 part of Dark rum, 1/2 part of Orgeat syrup, 2 parts of Orange juice, 1 part of Pineapple juice. Pour all ingredients, except the bitters, into shaker filled with ice. Shake well. Pour into large glass, filled with ice. Add Angostura bitters, ""on top"". Garnish with cocktail cherry and pineapple."
3,"This is called the Pink Penocha. 750 ml of Everclear, 1750 ml of Vodka, 1750 ml of Peach schnapps, 1 gal of Orange juice, 1 gal of Cranberry juice. mix all ingredients into bowl keep iced stir frequently"
4,"This is called the Paradise. 7 parts of Gin, 4 parts of Apricot Brandy, 3 parts of Orange Juice. Shake together over ice. Strain into cocktail glass and serve chilled."
...,...
541,"This is called the Fruit Shake. 1 cup fruit of Yoghurt, 1 of Banana, 4 oz frozen of Orange juice, 1/2 piece textural of Fruit, 6 of Ice. Blend til smooth."
542,"This is called the 151 Florida Bushwacker. 1/2 oz of Malibu rum, 1/2 oz of Light rum, 1/2 oz Bacardi of 151 proof rum, 1 oz of Dark Creme de Cacao, 1 oz of Cointreau, 3 oz of Milk, 1 oz of Coconut liqueur, 1 cup of Vanilla ice-cream. Combine all ingredients. Blend until smooth. Garnish with chocolate shavings if desired."
543,"This is called the 1-900-FUK-MEUP. 1/2 oz of Absolut Kurant, 1/4 oz of Grand Marnier, 1/4 oz of Chambord raspberry liqueur, 1/4 oz of Midori melon liqueur, 1/4 oz of Malibu rum, 1/4 oz of Amaretto, 1/2 oz of Cranberry juice, 1/4 oz of Pineapple juice. Shake ingredients in a mixing tin filled with ice cubes. Strain into a rocks glass."
544,"This is called the Frisco Sour. 2 oz of Blended whiskey, 1/2 oz of Benedictine, Juice of 1/4 of Lemon, Juice of 1/2 of Lime, 1 slice of Lemon, 1 slice of Lime. Shake all ingredients (except slices of lemon and lime) with ice and strain into a whiskey sour glass. Decorate with the slices of lemon and lime and serve."


## Train the synthetic model
In this step, we will task the worker running in the Gretel cloud, or locally, to fine-tune the GPT language model on the source dataset.

In [ ]:
%%time 

model = project.create_model_obj(model_config=config)
model.data_source = "training_data.csv"
model.name = "cocktail-generator"
model.submit_cloud()

poll(model)

INFO: Starting poller


{
    "uid": "6290f1085663d55145802234",
    "guid": "model_29knfrbpny0OHg8K0T76ICseSZh",
    "model_name": "cocktail-generator",
    "runner_mode": "cloud",
    "user_id": "625efa4ebff6215ff6745490",
    "user_guid": "user_281klalgbQOgeEQnzDYmRzHXKUu",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28eujAnf9EFme26oSFok8xCUT4n",
    "project_id": "6290f0f282cdc1de19b4cf92",
    "project_guid": "proj_29knczcGZtoGdlg4w3moVC1nWBZ",
    "status_history": {
        "created": "2022-05-27T15:40:56.842836Z"
    },
    "last_modified": "2022-05-27T15:40:57.039991Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:913274bb5193c7ede3a3cfc0cfc167b31774a4ca7c9831825a5205ea9d48157f",
    "model_type": "gpt_x",
    "config": {
        "schema_version": "1.0",
        "name": "cocktail-gen

INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2022-05-27T15:41:06.133827Z  Training gpt_x!
2022-05-27T15:41:34.114727Z  Training in progress
{
    "loss": 3.3905,
    "learning_rate": 6.500000000000001e-05,
    "epoch": 0.95
}
2022-05-27T15:41:34.756331Z  Training in progress
{
    "eval_loss": 2.781712770462036,
    "eval_runtime": 0.6386,
    "eval_samples_per_second": 170.689,
    "eval_steps_per_second": 10.962,
    "epoch": 0.95
}
2022-05-27T15:41:45.159285Z  Training in progress
{
    "loss": 2.3949,
    "learning_rate": 0.00013000000000000002,
    "epoch": 1.95
}
2022-05-27T15:41:45.801277Z  Training in progress
{
    "eval_loss": 2.3426568508148193,
    "eval_runtime": 0.6394,
    "eval_samples_per_second": 170.477,
    "eval_steps_per_second": 10.948,
    "epoch": 1.95
}
2022-05-27T15:41:57.161471Z  Training in 

CPU times: user 745 ms, sys: 97.3 ms, total: 842 ms
Wall time: 2min 17s


## Generate synthetic text data
You can now use the fine-tuned synthetic model to generate as much synthetic data as you like. The next cells walk through three ways to generate data.

1.  Generate text records from the model
2.  Generate text records using a single text seed (or prompt)
3.  Generate text records using a unique seed per record

In [ ]:
# Example 1: Generate text records from the model.
pd.set_option('display.max_rows', 20)

record_handler = model.create_record_handler_obj(
    params={"num_records": 20, "maximum_text_length": 128}
)
record_handler.submit_cloud()
poll(record_handler)

df = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
df

INFO: Starting poller


{
    "uid": "6290f24f40dd75b0367a1ce7",
    "guid": "model_run_29koKvLDbuWIvVD7lBB1haUgqTn",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "625efa4ebff6215ff6745490",
    "user_guid": "user_281klalgbQOgeEQnzDYmRzHXKUu",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28eujAnf9EFme26oSFok8xCUT4n",
    "project_id": "6290f0f282cdc1de19b4cf92",
    "project_guid": "proj_29knczcGZtoGdlg4w3moVC1nWBZ",
    "status_history": {
        "created": "2022-05-27T15:46:23.241000Z"
    },
    "last_modified": "2022-05-27T15:46:23.387000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:913274bb5193c7ede3a3cfc0cfc167b31774a4ca7c9831825a5205ea9d48157f",
    "model_id": "6290f1085663d55145802234",
    "model_guid": "model_29knfrbpny0OHg8K0T76ICseSZh",
    "action": "gpt

INFO: Status is created. A job has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated
INFO: Status is active. A worker has started!
2022-05-27T15:46:43.892888Z  Sampling 20 records conditioned on ''.
2022-05-27T15:46:45.806488Z  Generating 20 records
2022-05-27T15:46:54.443372Z  [2/20] records complete.
{
    "completed_count": 2,
    "total": 20
}
2022-05-27T15:47:02.953905Z  [4/20] records complete.
{
    "completed_count": 4,
    "total": 20
}
2022-05-27T15:47:11.481649Z  [6/20] records complete.
{
    "completed_count": 6,
    "total": 20
}
2022-05-27T15:47:19.966378Z  [8/20] records complete.
{
    "completed_count": 8,
    "total": 20
}
2022-05-27T15:47:28.470784Z  [10/20] records complete.
{
    "completed_count": 10,
    "total": 20
}
2022-05-27T15:47:37.008616Z  [12/20] records complete.
{
    "completed_count": 12,
    "total": 20
}
2022-05-27T15:47:45.557896Z  [14/20] records complete.
{
    "completed_count": 14,
    "total": 20
}
2022-05-27T15:4

,recipe
0,"is called the Long Island Cocktail. 3/4 oz of Vodka, 1/4 oz of Lime juice, of Ginger ale. Add ice and mix into glass over ice cubes. Add the remaining ingredients and stir. Strain into chilled cocktail glass. Garnish with lime wedge and ginger root. Serve in a double shot glass with a chilled pineapple juice. Garnish with lime wedge and ginger root. Serves two. Serves one shot. Serves one shot. Serves one shot. Serves one shot. Serves one shot. Serves two. Serves two. Serves two. Serves three."
1,"is called the Long Island Cocktail. 1 1/2 oz of Vodka, 2 oz of Dry Vermouth, 1/2 oz of Brandy, 1 dash of Triple sec, 1 oz of Ice. Shake well with a mixing mix. Strain into an old-fashioned glass over ice cubes. Garnish with orange and lime wedges. Top with the orange and lime juice. Serves six. Serves six, serving two. Serves two. Serves two. Serves two. Serves six. Serves two. Serves six. Serves six. Serves six. Serves six. Serves"
2,"is called the French Sazerac. 2 oz of Vodka, 2 oz of Grenadine, 1 of Cherry, 1 oz of Orange juice. Shake all ingredients with ice, strain into a cocktail glass, and serve. Garnish with a cherry and the orange slice, and serve with the lime wedge. If not chilled, garnish with the cherry and the orange slice and the cherry. Serve over ice and serve with the lime wedge. Serves over ice. Serves over ice, garnish with the cherry and the orange slice. Makes about 1-2 minutes. Garnish with the cherry and orange slice and the orange"
3,"is called the Rum. 1 part of Amaretto, 1/2 part of Galliano. Fill the rim of the glass with of Rum. Pour into an old-fashioned glass filled with ice cubes. Serve cold. Serve garnished with the shaved of shaved ice. Sprinkle the shaved of shaved ice with the shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of shaved of"
4,"is called the Red Queen. 1 1/2 oz of Vodka, 1/2 oz of Amaretto, Cream. Shake with ice. Strain into a cocktail glass. Decorate with a cream-on-a-stick and serve. Garnish with a cherry and serve. Serves 6. Makes 2 shots. Strain into a chilled glass and serve. Serves 12. Makes 4 shots. Makes 4 shots. Strain into a chilled glass. Serve in a punch bowl. Serves 6. Makes 2 shots. Strain into a chilled glass. Garnish with the cherry and serve. Serves 6"
5,"is called the Finsch Cowboy. 2 oz of Absolut Vodka, 1 oz of Chambordia, 1/2 oz of Orange juice, 1 oz of Light cream, 1 oz of Gin, 1/4 tsp of Lemon juice, 1 oz of Nutmeg, 1 twist of of Lime juice, 1 splash of of Sugar, Chocolate or Vanilla Ice. Stir vodka and chambordia with ice in a shaker almost filled with ice cubes. Shake absolut vodka and chambordia with nutmeg in a shaker almost filled with nutmeg-flour mixers. Strain and serve"
6,"is called the Jack and the Bean. 2 oz of Scotch, 2 oz of Light cream, Lemon peel. Stir all ingredients in an old fashioned coffee cup with ice, strain into a cocktail shaker and serve. Top with the rest of the drink and serve. Serves six, so you'll have a nice cold drink. 2 oz of Kahlua, 3 oz of Amaretto, 2 oz of Orange juice, Orange cream. Shake all ingredients with ice and strain into a cocktail shaker and serve. Serves six, so you'll have a nice cold drink. 3 oz"
7,"is called the Apello. 1 oz of Cointreau, 1 oz of Brandy, 1/2 oz of Orange juice. In a shaker half-filled with ice cubes, combine all of the ingredients. Shake well. Strain into a whiskey sour glass. Add the brandy and serve. Garnish with a cherry. Makes about 1 bottle. If you want to make an orange juice glass you may do this. Garnish with the orange slice. Orange Juice 1 oz Orange Juice 1 oz Juice 1 oz Orange Juice 1 oz 1/2 oz of Brandy. In a sh"
8,"is called the Vodka Vodka. 1 oz of Soda water, 1 oz of Water, 1/2 oz of Gin, 1 oz of Orange juice, 1 oz of

In [ ]:
# Example 2: Generate text with an optional "prompt" which is used to condition
# model generation.

record_handler = model.create_record_handler_obj(
    params={"num_records": 5, 
            "maximum_text_length": 128,
            "prompt": "A software engineer walks into a bar and wants to try something new. What should they order?"}
)
record_handler.submit_cloud()
poll(record_handler)

df = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
df

INFO: Starting poller


{
    "uid": "6290f40f35d6034324f09ad2",
    "guid": "model_run_29kpFCNadCbSYKuX97bw33qtu65",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "625efa4ebff6215ff6745490",
    "user_guid": "user_281klalgbQOgeEQnzDYmRzHXKUu",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28eujAnf9EFme26oSFok8xCUT4n",
    "project_id": "6290f0f282cdc1de19b4cf92",
    "project_guid": "proj_29knczcGZtoGdlg4w3moVC1nWBZ",
    "status_history": {
        "created": "2022-05-27T15:53:51.233000Z"
    },
    "last_modified": "2022-05-27T15:53:51.372000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:913274bb5193c7ede3a3cfc0cfc167b31774a4ca7c9831825a5205ea9d48157f",
    "model_id": "6290f1085663d55145802234",
    "model_guid": "model_29knfrbpny0OHg8K0T76ICseSZh",
    "action": "gpt

INFO: Status is created. A job has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated
INFO: Status is active. A worker has started!
2022-05-27T15:54:13.150367Z  Sampling 5 records conditioned on 'A software engineer walks into a bar and wants to try something new. What should they order?'.
2022-05-27T15:54:15.040157Z  Generating 5 records
2022-05-27T15:54:37.362125Z  Successfully generated 5 records
2022-05-27T15:54:37.387757Z  Uploading artifacts to Gretel Cloud


,recipe
0,"First they order A Russian vodka shot, then a Swedish coffee. Then they order A Russian cherry, then a Swedish cup of Coffee. Then they order A Russian and then a Swedish, then another. Then they do A Swedish and then a Russian. The vodka is served in a glass and then the cherry on top. Then they have a shot of Russian vodka and then a shot of Swedish coffee. Then the shot is taken on the rocks. Then the coffee is put in and then they order the Russian. Then the shot is served in a shot glass. Then they put the shot on top of the rocks and then order the coffee."
1,"Pour the wine, dip in the salt, top with a dash of of Bitters, pour the rest of the way over the dash. Then serve at a highball. Pour the rest of the way. Add the crushed ice cubes and top off with the dash of bitters. Serve cold. Just stir occasionally and the drink tastes better. Makes 10. 5 oz of Vodka, 4 oz of Gin, 1/2 oz of Brandy, 1 of Triple sec, 1 twist of Lemon peel. Garnish with the bitters and the lime wedge. Makes 12 servings. Makes 1 pint. Makes 4 shots. Makes"
2,"A cold beer, 1 oz of Gin, 1 oz of Rum, 1 dash of Triple sec. Shake with a little ice and strain into a cocktail glass. Shake with a little sugar and drink. You'll like it! It makes the drink really old fashioned and very real. This will definitely improve the mood. 1 oz of Gin, 1 oz of Brandy, 1 oz of Apricot brandy, 1 dash of Triple sec. Shake with ice and strain into a chilled cocktail glass. Enjoy! Enjoy! (can never be drunk, it's called the Gin!) - - - - - - - - - - - -"
3,"2 oz of Gin, 2 oz of Triple sec, 2 tsp of Pineapple juice, 1/2 tsp of Lemon juice, 1/2 tsp of Dry Vermouth, 2 tsp of Orange juice, 1/2 tsp of Powdered sugar, 1 of Egg white, 1 of Orange, 1 of Orange, 1 of Green Creme de Cacao, 1 of Orange, 1 of Orange, 2 of Galliano. Shake all ingredients with ice, strain into a cocktail glass, and serve. Not too hot, fill with rum, shake, and serve. If preferred, make the sugar syrup to resemble the name"
4,"Chambord. 1 shot of Tequila, 1 shot of Vodka. Stir ingredients into mixing bowl and add ice cubes. Stir and pour into a shot glass. Strain into a cocktail glass and serve in a large glass in a large martini glass. (This is called the Aperol Vodka. 1 part of Vodka, 1 part of Tequila.) Serve in chilled wine glass or cocktail glass with garnish. Decorate with a pineapple and a pineapple slice. And never, never again drink a shot glass without pineapple. 2. 1/4 shot of Tequila, 1 part of Vodka, 1 part"


In [ ]:
# Example 3: Generate text with optional "prompts" to condition model generation 
# with an individual prompt for each record in CSV format.

INGREDIENTS = ('orange juice', 'lime', 'lemon juice', 'Bourbon', 'Gin')

prompts = pd.DataFrame([f'Can you make me a drink with {ingredient} in it?' 
                        for ingredient in INGREDIENTS], columns=["text"])
prompts.to_csv('prompts.csv', index=False)

record_handler = model.create_record_handler_obj(
    params={"maximum_text_length": 128},
    data_source='prompts.csv'
)
record_handler.submit_cloud()
poll(record_handler)

df = pd.read_csv(record_handler.get_artifact_link("data"), compression='gzip')
df

INFO: Starting poller


{
    "uid": "6290f54135d6034324f09ad3",
    "guid": "model_run_29kprhxg3ffdY4WOGjrrEXKzgZV",
    "model_name": null,
    "runner_mode": "cloud",
    "user_id": "625efa4ebff6215ff6745490",
    "user_guid": "user_281klalgbQOgeEQnzDYmRzHXKUu",
    "billing_domain": "gretel.ai",
    "billing_domain_guid": "domain_28eujAnf9EFme26oSFok8xCUT4n",
    "project_id": "6290f0f282cdc1de19b4cf92",
    "project_guid": "proj_29knczcGZtoGdlg4w3moVC1nWBZ",
    "status_history": {
        "created": "2022-05-27T15:58:57.791000Z"
    },
    "last_modified": "2022-05-27T15:58:57.910000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/gpt_x@sha256:913274bb5193c7ede3a3cfc0cfc167b31774a4ca7c9831825a5205ea9d48157f",
    "model_id": "6290f1085663d55145802234",
    "model_guid": "model_29knfrbpny0OHg8K0T76ICseSZh",
    "action": "gpt

INFO: Status is created. A job has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated
INFO: Status is active. A worker has started!
2022-05-27T15:59:20.364219Z  Sampling 5 records using conditioning input...
2022-05-27T15:59:22.236450Z  Generating 5 records
2022-05-27T15:59:43.960787Z  Successfully generated 5 records
2022-05-27T15:59:43.985039Z  Uploading artifacts to Gretel Cloud


,recipe
0,"2 cl of Amaretto, 1 cl of Peach schnapps, Juice of 1/2 of Lemon, 1 tsp of Sugar. Shake with ice, strain into a cocktail glass, and serve. If you really want to make the drink, add one of the brandy. Then you are done. Pour the drink into your order and leave for a day or so. If you prefer a tall, dark, and dark stout, you can make it without a drink. But you should have 1/2 of Lemon juice, 1/4 of Ginger, 1/4 of Apple. Juice all of the ingredients. Shake with a"
1,"1 1/2 oz of Dry Vermouth, 1 1/2 oz of Cointreau, 1 oz of Dry Vermouth, 1 of Cranberry juice. Mix all ingredients in glass with ice and serve. Makes 1 drink. Pour 1/2 oz of Dry Vermouth into glass, stir, and serve. Serves two and gives one or two to top with a splash of Cointreau, a slice of Cranberry juice, and a twist of lime. Serves one at a time. 1 1/2 oz of Dry Vermouth, 1 oz of Cointreau. Pour all ingredients into glass. Serves one"
2,"In my head, I'll drink a lemon shot, 1 oz of Gin, 1 oz of Brandy, 1/2 oz of Grenadine, 1/2 oz of Orange juice, 1 dash of Bitters, 1 dash of Orange juice, 1 dash of Orange brandy. In the glass of a cocktail shaker, put lemon juice and grenadine. Shake together with ice and serve with a straw. Or use the dash of orange juice. A shot of liqueur and some gin, 1 1/2 oz of Grenadine, 1 oz of Orange juice, 1/2 oz of Apple brandy,"
3,"2 cups of Beer, 1 1/2 tsp of Sugar, 1 twist of of Cointreau, 1 dash of Triple Sec, 1 slice of Lime, 1 of Cherry, 1 dash of Angostura spread on top, 1 of Green Vodka, 1 slice of Orange. Shake all ingredients with ice and strain into a tall glass. Add all contents into a highball glass. Garnish with lime slices and orange slices and a dash of Angostura and a slice of lime, if desired. Enjoy! Enjoy!!! Creme de Cacao 2 oz of Cointreau, Grapefruit"
4,"1 of Lime, 1/4 of Gin, 1 of Orange, 1 of Orange, 1 slice of of Orange, 1 slice of of Orange, 1 of Lemon, 1 slice of of Lemon, 1 of Cherry, 1 slice of Cherry, 1 slice of Cointreau, 1 slice of Cointreau. In a shaker half-filled with ice cubes, combine the lime, gin, and orange juice. Shake well. (To get the drink, combine the orange slice with the lime and lime zest. Don't let the slice get too dense!) Strain into a chilled cocktail glass and serve at once. Makes 4"
